In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import os
import docx
import re

<IPython.core.display.Javascript object>

In [3]:
path_docs = "../data/Документы/"
docs_name = [elem for elem in os.listdir(path_docs) if elem.split(".")[-1] == "docx"]
len(docs_name)

165

<IPython.core.display.Javascript object>

## full

In [4]:
def get_text(doc):
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text.strip())
    fullText = [elem for elem in fullText if len(elem) > 0]
    return fullText

<IPython.core.display.Javascript object>

In [5]:
def get_first_index(fullText):
    ind_start = 0
    for i in range(len(fullText) - 1):
        if ("правила" in fullText[i].lower()) and (
            "предоставления" in fullText[i + 1].lower()
        ):
            ind_start = i
            break
    return ind_start

<IPython.core.display.Javascript object>

In [6]:
def get_end_index(fullText):
    end_start = len(fullText)
    for i in range(ind_start, len(fullText) - 1):
        if (
            (
                fullText[i].lower().startswith("утвержден")
                & fullText[i + 1].lower().startswith("постановлением правительства")
            )
            | (
                ("приложение №" in fullText[i].lower())
                & ("предоставления" in fullText[i + 1].lower())
            )
            | (
                ("приложение №" in fullText[i].lower())
                & ("предоставления" in fullText[i].lower())
            )
        ):
            #         if (i < end_start - 6) & ("правила" in fullText[i + 4].lower()) and (
            #             "предоставления" in fullText[i + 5].lower()
            #         ):
            #             continue
            if i < len(fullText) - 15:
                if (
                    (len(re.findall("\{\d+\}", fullText[i + 10])) > 0)
                    | (len(re.findall("\{\d+\}", fullText[i + 12])) > 0)
                    | (len(re.findall("\{\d+\}", fullText[i + 14])) > 0)
                ):
                    continue
            end_start = i
            break
    return end_start

<IPython.core.display.Javascript object>

In [100]:
len(
    "Настоящие Правила устанавливают цели, порядок и условия предоставления субсидий из федерального бюджета организации, наделенной Правительством Российской Федерации функциями оператора в части финансового обеспечения реализации некоммерческими организациями, осуществляющими функции инфраструктурных центров, программ по развитию отдельных направлений"
)

350

<IPython.core.display.Javascript object>

In [7]:
full_docs = dict()
cnt_errors = 0
file_w_errors = set()
for cur_doc in docs_name:
    filename = path_docs + cur_doc
    doc = docx.Document(filename)

    fullText = get_text(doc)

    # ищем начало раздела Правила предоставления ...
    ind_start = get_first_index(fullText)

    # ищем конец раздела Правила предоставления ...
    end_start = get_end_index(fullText)

    target_data = []
    activate_flag = False
    for elem in fullText[ind_start:end_start]:
        if not activate_flag:
            if (
                len(re.findall("\{\d+\}", elem)) > 0
            ):  # if len(re.findall("\d. \{\d\}", elem)) > 0:
                activate_flag = True
                target_data.append(elem.replace("\xa0", " "))
        else:
            target_data.append(elem.replace("\xa0", " "))
    target_data = [elem for elem in target_data if "Правительства РФ от" not in elem]
    # извлекаем класс и текст
    texts = []
    targets = []
    few_texts = []
    few_targets = []
    real_target_data = []
    for num, elem in enumerate(target_data):
        elem_old = elem
        ## one mode
        elem = re.sub("{\d+}\s{0,3}{\d+}", "", elem)
        if len(re.split("{\d+}", elem)) > 1:
            target = int(re.split("{", elem)[1].split("}")[0])
            text = re.split("{\d+}", elem)[1].strip()
        else:
            target = 0
            text = elem.strip()
        ## few mode
        re.findall("{\d+}", elem_old)
        few_target = re.findall("{\d+}", elem_old)[::2]
        few_target = [int(re.split("{", x)[1].split("}")[0]) for x in few_target]
        few_text = re.split("{\d+}.{0,5}{\d+}", elem_old)
        few_text = [re.sub("{\d+}", "", line) for line in few_text]
        if len(few_target) == 0:
            few_target = [0 for _ in few_text]
        ### ПРОВЕРКИ ОШИБОК в targts
        some_trgets = re.findall("{\d+}|{\d+\)", elem_old)
        mistake_flag = False
        for i in range(0, len(some_trgets), 2):
            try:
                if some_trgets[i] != some_trgets[i + 1]:
                    mistake_flag = True
                    print(
                        "ERROR pair tags not equal | filename =", cur_doc, some_trgets
                    )
            except:
                mistake_flag = True
                print("ERROR нечетное к-во some_trgets", cur_doc, some_trgets)
        # если нет ошибок в данных,то учитываем
        if not mistake_flag:
            texts.append(text)
            targets.append(target)
            few_texts.append(few_text)
            few_targets.append(few_target)
            real_target_data.append(elem_old)
    # инициализируем все в датасет
    name = cur_doc.split(".")[0]
    full_docs[name] = dict()
    full_docs[name]["data"] = real_target_data  # target_data
    full_docs[name]["text"] = texts
    full_docs[name]["target"] = targets
    full_docs[name]["few_texts"] = few_texts
    full_docs[name]["few_targets"] = few_targets

    ### ПРОВЕРКИ ВОЗМОЖНЫХ ОШИБОК
    # check mistake END
    if end_start < len(fullText):
        for num, elem in enumerate(fullText[end_start:]):
            if len(re.findall("\{\d+\}", elem)) > 0:
                print(cur_doc)
                print("END ERROr", num + end_start, elem)
                print(10 * "---")
                cnt_errors += 1
                file_w_errors.add(cur_doc)
    # check mistake START
    if ind_start == 0:
        print(cur_doc)
        print("START ERROr:стартуем с самого начала")
        print(10 * "---")
        cnt_errors += 1
        file_w_errors.add(cur_doc)
    elif ind_start < len(fullText):
        for num, elem in enumerate(fullText[:ind_start]):
            if len(re.findall("\{\d+\}", elem)) > 0:
                print(cur_doc)
                print("START ERROr", num, elem)
                print(20 * "===")
                cnt_errors += 1
                file_w_errors.add(cur_doc)

ERROR pair tags not equal | filename = Postanovlenie_Pravitelstva_RF_ot_31_03_2021_N_487.docx ['{22}', '{21}']
ERROR pair tags not equal | filename = Postanovlenie_Pravitelstva_Rossiyskoy_Federatsii_ot_04_06_2020_823.docx ['{15}', '{35}', '{35}', '{35}', '{35}', '{15}']
ERROR pair tags not equal | filename = Postanovlenie_Pravitelstva_Rossiyskoy_Federatsii_ot_04_06_2020_823.docx ['{15}', '{35}', '{35}', '{35}', '{35}', '{15}']
ERROR pair tags not equal | filename = Postanovlenie_Pravitelstva_Rossiyskoy_Federatsii_ot_19_05_2021_759.docx ['{28}', '{3}', '{3}', '{28}']
ERROR pair tags not equal | filename = Postanovlenie_Pravitelstva_Rossiyskoy_Federatsii_ot_19_05_2021_759.docx ['{28}', '{3}', '{3}', '{28}']
ERROR pair tags not equal | filename = Постановление Правительства Российской Федерации от 14.04.2022 № 662.docx ['{37}', '{7}']
ERROR pair tags not equal | filename = Постановление Правительства РФ от 28.06.2021 N 1031.docx ['{10}', '{24}']
ERROR pair tags not equal | filename = Пост

<IPython.core.display.Javascript object>

In [8]:
cnt_errors

0

<IPython.core.display.Javascript object>

In [9]:
len_errors = len(file_w_errors)
if len_errors < 10:
    print(file_w_errors)
else:
    print(len_errors)

set()


<IPython.core.display.Javascript object>

In [10]:
# full_docs = dict()
# for cur_doc in docs_name:
#     filename = path_docs + cur_doc
#     doc = docx.Document(filename)

#     # считаем все данные
#     fullText = []
#     for para in doc.paragraphs:
#         fullText.append(para.text.strip())
#     # убираем пустые строки
#     fullText = [elem for elem in fullText if len(elem) > 0]

#     # ищем начало раздела Правила предоставления ...
#     ind_start = 0
#     real_text = []
#     for i in range(len(fullText) - 1):
#         if fullText[i].lower().startswith("правила") and fullText[
#             i + 1
#         ].lower().startswith("предоставления"):
#             ind_start = i
#             break

#     # ищем конец раздела Правила предоставления ...
#     end_start = len(fullText)
#     for i in range(ind_start, len(fullText) - 1):
#         if (
#             (
#                 fullText[i].lower().startswith("утвержден")
#                 & fullText[i + 1].lower().startswith("постановлением правительства")
#             )
#             | (
#                 ("приложение №" in fullText[i].lower())
#                 & ("предоставления" in fullText[i + 1].lower())
#             )
#             | (
#                 ("приложение №" in fullText[i].lower())
#                 & ("предоставления" in fullText[i].lower())
#             )
#         ):
#             end_start = i
#             break

#     target_data = []
#     activate_flag = False
#     for elem in fullText[ind_start:end_start]:
#         if not activate_flag:
#             if (
#                 len(re.findall("\{\d\}", elem)) > 0
#             ):  # if len(re.findall("\d. \{\d\}", elem)) > 0:
#                 activate_flag = True
#                 target_data.append(elem.replace("\xa0", " "))
#         else:
#             target_data.append(elem.replace("\xa0", " "))
#     target_data = [elem for elem in target_data if "Правительства РФ от" not in elem]
#     ### ПРОВЕРКИ ВОЗМОЖНЫХ ОШИБОК
#     # check mistake END
#     if end_start < len(fullText):
#         for num, elem in enumerate(fullText[end_start:]):
#             if len(re.findall("\{\d\}", elem)) > 0:
#                 print(cur_doc)
#                 print("END ERROr", num + end_start, elem)
#                 print(10 * "---")
#     # check mistake START
#     if ind_start == 0:
#         print(cur_doc)
#         print("START ERROr:стартуем с самого начала")
#         print(10 * "---")
#     elif ind_start < len(fullText):
#         for num, elem in enumerate(fullText[:ind_start]):
#             if len(re.findall("\{\d\}", elem)) > 0:
#                 print(cur_doc)
#                 print("START ERROr", num, elem)
#                 print(20 * "===")
#     # извлекаем класс и текст
#     texts = []
#     targets = []
#     for num, elem in enumerate(target_data):
#         elem = re.sub("{\d+}\s{0,3}{\d+}", "", elem)
#         if len(re.split("{\d+}", elem)) > 1:
#             target = int(re.split("{", elem)[1].split("}")[0])
#             text = re.split("{\d+}", elem)[1].strip()
#         else:
#             target = 0
#             text = elem.strip()
#         texts.append(text)
#         targets.append(target)
#     # инициализируем все в датасет
#     name = cur_doc.split(".")[0]
#     full_docs[name] = dict()
#     full_docs[name]["data"] = target_data
#     full_docs[name]["text"] = texts
#     full_docs[name]["target"] = targets

<IPython.core.display.Javascript object>

In [11]:
len(full_docs)

156

<IPython.core.display.Javascript object>

In [12]:
# full_docs["Postanovlenie_Pravitelstva_Rossiyskoy_Federatsii_ot_22_03_2021_439"]

<IPython.core.display.Javascript object>

In [13]:
# full_docs["955_ot_27_05_2022"]

<IPython.core.display.Javascript object>

In [14]:
import json

with open("full_docs.json", "w+") as fp:
    json.dump(full_docs, fp)

<IPython.core.display.Javascript object>

In [15]:
with open("full_docs.json", "r+") as fp:
    full_docs1 = json.load(fp)

<IPython.core.display.Javascript object>

In [16]:
key_ = list(full_docs1.keys())[0]
key_

'Postanovlenie_Pravitelstva_RF_ot_29_09_2017_N_1179'

<IPython.core.display.Javascript object>

In [17]:
full_docs1[key_].keys()

dict_keys(['data', 'text', 'target', 'few_texts', 'few_targets'])

<IPython.core.display.Javascript object>

In [19]:
import numpy as np

<IPython.core.display.Javascript object>

In [20]:
for key_ in full_docs1.keys():
    pp = np.array(
        [
            len(full_docs1[key_]["data"]),
            len(full_docs1[key_]["text"]),
            len(full_docs1[key_]["target"]),
            len(full_docs1[key_]["few_texts"]),
            len(full_docs1[key_]["few_targets"]),
        ]
    )
    check = np.where(pp != len(full_docs1[key_]["data"]))[0]
    if len(check) > 0:
        print(key_, pp)

<IPython.core.display.Javascript object>

## for one

In [67]:
cur_doc = "Postanovlenie_pravitelstva_RF_02_03_2021_N_294.docx"
cur_doc = "Postanovlenie_Pravitelstva_RF_ot_31_03_2021_N_487.docx"
# ошибка разметки:  7. {22} Размер субсидии (S) по направлениям ее использования рассчитывается по формуле: {21}
cur_doc = "Постановление Правительства РФ от 28.06.2021 N 1031.docx"
# 66 {10}доменное имя, и (или) сетевой адрес, и (или) указатели страниц сайта в информационно-телекоммуникационной сети "Интернет", на котором обеспечивается проведение отбора; {24}
# 75 {15} обязательство Фонда о проведении отбора в соответствии с порядком, установленным  настоящих Правил; {24}
# 77 {27}значения показателей, необходимых для достижения результата предоставления субсидии, указанные в  настоящих Правил; {24}

# cur_doc = "804_ot_30_aprelya_2022.docx"
# вместо {11} ->{11)

# cur_doc = "574_ot_2_aprelya_2022.docx"
cur_doc = "Постановление Правительства РФ от 28.06.2021 N 1031.docx"

<IPython.core.display.Javascript object>

In [68]:
# файлы, на которых были ошибки
# теперь пришлось вперед смотреть для проверке (НА ТРЕЙНЕ ТАК НЕЛЬЗЯ), надо будет уточнить у кейсодеражетелей
# cur_doc = "Постановление Правительства РФ от 03.11.2018 N 1304.docx"
# cur_doc = "Postanovlenie_Pravitelstva_Rossiyskoy_Federatsii_ot_22_03_2021_439.docx"
# cur_doc = "Postanovlenie_Pravitelstva_RF_ot_03_04_2018_N_402.docx"

<IPython.core.display.Javascript object>

In [69]:
# !pip install python-docx

<IPython.core.display.Javascript object>

In [70]:
filename = path_docs + cur_doc
doc = docx.Document(filename)

<IPython.core.display.Javascript object>

In [71]:
fullText = get_text(doc)

<IPython.core.display.Javascript object>

In [72]:
# fullText = []
# for para in doc.paragraphs:
#     fullText.append(para.text.strip())

<IPython.core.display.Javascript object>

In [73]:
# fullText = [elem for elem in fullText if len(elem) > 0]
for num, elem in enumerate(fullText):
    print(num, elem)

0 ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ
1 ПОСТАНОВЛЕНИЕ
2 от 28 июня 2021 г. N 1031
3 ОБ УТВЕРЖДЕНИИ ПРАВИЛ
4 ПРЕДОСТАВЛЕНИЯ СУБСИДИИ ИЗ ФЕДЕРАЛЬНОГО БЮДЖЕТА РОССИЙСКОМУ
5 ФОНДУ РАЗВИТИЯ ИНФОРМАЦИОННЫХ ТЕХНОЛОГИЙ НА ВОЗМЕЩЕНИЕ
6 ЗАТРАТ ПО ИСПОЛЬЗОВАНИЮ СУБЪЕКТАМИ МАЛОГО И СРЕДНЕГО
7 ПРЕДПРИНИМАТЕЛЬСТВА РОССИЙСКОГО ПРОГРАММНОГО ОБЕСПЕЧЕНИЯ
8 Правительство Российской Федерации постановляет:
9 Утвердить прилагаемые  предоставления субсидии из федерального бюджета Российскому фонду развития информационных технологий на возмещение затрат по использованию субъектами малого и среднего предпринимательства российского программного обеспечения.
10 Председатель Правительства
11 Российской Федерации
12 М.МИШУСТИН
13 Утверждены
14 постановлением Правительства
15 Российской Федерации
16 от 28 июня 2021 г. N 1031
17 ПРАВИЛА
18 ПРЕДОСТАВЛЕНИЯ СУБСИДИИ ИЗ ФЕДЕРАЛЬНОГО БЮДЖЕТА
19 РОССИЙСКОМУ ФОНДУ РАЗВИТИЯ ИНФОРМАЦИОННЫХ ТЕХНОЛОГИЙ
20 НА ВОЗМЕЩЕНИЕ ЗАТРАТ ПО ИСПОЛЬЗОВАНИЮ СУБЪЕКТАМИ МАЛОГО
21 И СРЕДНЕГО ПР

<IPython.core.display.Javascript object>

In [74]:
def get_first_index(fullText):
    ind_start = 0
    for i in range(len(fullText) - 1):
        if ("правила" in fullText[i].lower()) and (
            "предоставления" in fullText[i + 1].lower()
        ):
            ind_start = i
            break
    return ind_start

<IPython.core.display.Javascript object>

In [75]:
ind_start = get_first_index(fullText)

<IPython.core.display.Javascript object>

In [76]:
# ind_start = 0
# for i in range(len(fullText) - 1):
#     if ("правила" in fullText[i].lower()) and (
#         "предоставления" in fullText[i + 1].lower()
#     ):
#         ind_start = i
#         break

<IPython.core.display.Javascript object>

In [77]:
ind_start

17

<IPython.core.display.Javascript object>

In [78]:
"предоставления".lower()

'предоставления'

<IPython.core.display.Javascript object>

In [79]:
def get_end_index(fullText):
    end_start = len(fullText)
    for i in range(ind_start, len(fullText) - 1):
        if (
            (
                fullText[i].lower().startswith("утвержден")
                & fullText[i + 1].lower().startswith("постановлением правительства")
            )
            | (
                ("приложение №" in fullText[i].lower())
                & ("предоставления" in fullText[i + 1].lower())
            )
            | (
                ("приложение №" in fullText[i].lower())
                & ("предоставления" in fullText[i].lower())
            )
        ):
            #         if (i < end_start - 6) & ("правила" in fullText[i + 4].lower()) and (
            #             "предоставления" in fullText[i + 5].lower()
            #         ):
            #             continue
            if i < len(fullText) - 15:
                if (len(re.findall("\{\d+\}", fullText[i + 10])) > 0) & (
                    len(re.findall("\{\d+\}", fullText[i + 12])) > 0
                ):
                    continue
            end_start = i
            break
    return end_start

<IPython.core.display.Javascript object>

In [80]:
end_start = get_end_index(fullText)

<IPython.core.display.Javascript object>

In [81]:
end_start = len(fullText)
for i in range(ind_start, len(fullText) - 1):
    if (
        (
            fullText[i].lower().startswith("утвержден")
            & fullText[i + 1].lower().startswith("постановлением правительства")
        )
        | (
            ("приложение №" in fullText[i].lower())
            & ("предоставления" in fullText[i + 1].lower())
        )
        | (
            ("приложение №" in fullText[i].lower())
            & ("предоставления" in fullText[i].lower())
        )
        | (
            ("приложение N" in fullText[i].lower())
            & ("предоставления" in fullText[i].lower())
        )
        | (
            ("приложение N" in fullText[i].lower())
            & ("предоставления" in fullText[i + 1].lower())
        )
    ):
        #         if (i < end_start - 6) & ("правила" in fullText[i + 4].lower()) and (
        #             "предоставления" in fullText[i + 5].lower()
        #         ):
        #             continue
        if i < len(fullText) - 15:
            if (len(re.findall("\{\d+\}", fullText[i + 10])) > 0) | (
                len(re.findall("\{\d+\}", fullText[i + 12])) > 0
            ):
                continue
        end_start = i
        break

<IPython.core.display.Javascript object>

In [82]:
end_start, len(fullText)

(235, 235)

<IPython.core.display.Javascript object>

в каких разделах у нас мб текст:
 - РЕАЛИЗАЦИИ НЕКОММЕРЧЕСКИМИ ОРГАНИЗАЦИЯМИ ..., ОСУЩЕСТВЛЯЮЩИМИ ФУНКЦИИ ИНФРАСТРУКТУРНЫХ ЦЕНТРОВ, ПРОГРАММ ПО РАЗВИТИЮ ОТДЕЛЬНЫХ НАПРАВЛЕНИЙ НАЦИОНАЛЬНОЙ ТЕХНОЛОГИЧЕСКОЙ ИНИЦИАТИВЫ
 - ПРАВИЛА ... ФИНАНСОВОГО ОБЕСПЕЧЕНИЯ (ВОЗМЕЩЕНИЯ) ЗАТРАТ СПЕЦИАЛИЗИРОВАННЫХ ОБЩЕСТВ ПРОЕКТНОГО ФИНАНСИРОВАНИЯ ПО ВЫПЛАТЕ ПРОЦЕНТНОГО (КУПОННОГО) ДОХОДА ПО ОБЛИГАЦИЯМ
 - ПРАВИЛА ПРЕДОСТАВЛЕНИЯ СУБСИДИЙ 

In [83]:
target_data = []

activate_flag = False
for elem in fullText[ind_start:end_start]:
    if not activate_flag:
        if len(re.findall("\{\d\}", elem)) > 0:
            activate_flag = True
            target_data.append(elem.replace("\xa0", " "))
    else:
        target_data.append(elem.replace("\xa0", " "))

<IPython.core.display.Javascript object>

In [84]:
target_data

['1. {2} Настоящие Правила устанавливают цели, условия и порядок предоставления субсидии из федерального бюджета Российскому фонду развития информационных технологий на возмещение затрат по использованию субъектами малого и среднего предпринимательства российского программного обеспечения (далее соответственно - Фонд, субсидия). {2}',
 '{2}Субсидия предоставляется в рамках реализации федерального  "Цифровые технологии" национальной  "Цифровая экономика Российской Федерации".{2}',
 '2. {3}Субсидия предоставляется Министерством цифрового развития, связи и массовых коммуникаций Российской Федерации в пределах лимитов бюджетных обязательств, доведенных в установленном порядке до Министерства цифрового развития, связи и массовых коммуникаций Российской Федерации как получателя средств федерального бюджета на цели, указанные в  настоящих Правил. {3}',
 '{7}Сведения о субсидии размещаются на едином портале бюджетной системы Российской Федерации в информационно-телекоммуникационной сети "Интер

<IPython.core.display.Javascript object>

In [85]:
[elem for elem in target_data if "Правительства РФ от" not in elem]

['1. {2} Настоящие Правила устанавливают цели, условия и порядок предоставления субсидии из федерального бюджета Российскому фонду развития информационных технологий на возмещение затрат по использованию субъектами малого и среднего предпринимательства российского программного обеспечения (далее соответственно - Фонд, субсидия). {2}',
 '{2}Субсидия предоставляется в рамках реализации федерального  "Цифровые технологии" национальной  "Цифровая экономика Российской Федерации".{2}',
 '2. {3}Субсидия предоставляется Министерством цифрового развития, связи и массовых коммуникаций Российской Федерации в пределах лимитов бюджетных обязательств, доведенных в установленном порядке до Министерства цифрового развития, связи и массовых коммуникаций Российской Федерации как получателя средств федерального бюджета на цели, указанные в  настоящих Правил. {3}',
 '{7}Сведения о субсидии размещаются на едином портале бюджетной системы Российской Федерации в информационно-телекоммуникационной сети "Интер

<IPython.core.display.Javascript object>

In [86]:
# check mistake END
if end_start < len(fullText):
    for num, elem in enumerate(fullText[end_start:]):
        if len(re.findall("\{\d+\}", elem)) > 0:
            print(num + end_start, elem)

<IPython.core.display.Javascript object>

In [87]:
# check mistake START
if ind_start == 0:
    print("стартуем с самого начала")
elif ind_start < len(fullText):
    for num, elem in enumerate(fullText[:ind_start]):
        if len(re.findall("\{\d+\}", elem)) > 0:
            print(num + end_start, elem)

<IPython.core.display.Javascript object>

In [88]:
for num, line in enumerate(target_data):
    print(num, line)

0 1. {2} Настоящие Правила устанавливают цели, условия и порядок предоставления субсидии из федерального бюджета Российскому фонду развития информационных технологий на возмещение затрат по использованию субъектами малого и среднего предпринимательства российского программного обеспечения (далее соответственно - Фонд, субсидия). {2}
1 {2}Субсидия предоставляется в рамках реализации федерального  "Цифровые технологии" национальной  "Цифровая экономика Российской Федерации".{2}
2 2. {3}Субсидия предоставляется Министерством цифрового развития, связи и массовых коммуникаций Российской Федерации в пределах лимитов бюджетных обязательств, доведенных в установленном порядке до Министерства цифрового развития, связи и массовых коммуникаций Российской Федерации как получателя средств федерального бюджета на цели, указанные в  настоящих Правил. {3}
3 {7}Сведения о субсидии размещаются на едином портале бюджетной системы Российской Федерации в информационно-телекоммуникационной сети "Интернет" п

<IPython.core.display.Javascript object>

In [89]:
target_data[114]

'{27}количество мероприятий по популяризации приобретения субъектами малого и среднего предпринимательства российского программного обеспечения по льготной цене. {27}'

<IPython.core.display.Javascript object>

In [90]:
texts = []
targets = []
few_texts = []
few_targets = []
real_target_data = []
for num, elem in enumerate(target_data):
    elem_old = elem
    ## one mode
    elem = re.sub("{\d+}\s{0,3}{\d+}", "", elem)
    if len(re.split("{\d+}", elem)) > 1:
        target = int(re.split("{", elem)[1].split("}")[0])
        text = re.split("{\d+}", elem)[1].strip()
    else:
        target = 0
        text = elem.strip()
    texts.append(text)
    targets.append(target)
    ## few mode
    few_target = re.findall("{\d+}", elem_old)[::2]
    few_target = [int(re.split("{", x)[1].split("}")[0]) for x in few_target]
    few_text = re.split("{\d+}.{0,5}{\d+}", elem_old)
    few_text = [re.sub("{\d+}|{\d+\)", "", line) for line in few_text]
    if len(few_target) == 0:
        few_target = [0 for _ in few_text]
    few_texts.append(few_text)
    few_targets.append(few_target)
    ### ПРОВЕРКИ ОШИБОК в targts
    some_trgets = re.findall("{\d+}|{\d+\)", elem_old)
    mistake_flag = False
    for i in range(0, len(some_trgets), 2):
        try:
            if some_trgets[i] != some_trgets[i + 1]:
                mistake_flag = True
                print("ERROR pair tags not equal | num_doc =", num, some_trgets)
        except:
            mistake_flag = True
            print("ERROR нечетное к-во some_trgets", num, some_trgets)
    # если нет ошибок в данных,то учитываем
    if not mistake_flag:
        texts.append(text)
        targets.append(target)
        few_texts.append(few_text)
        few_targets.append(few_target)
        real_target_data.append(elem_old)

    if num == 114:
        break

ERROR pair tags not equal | num_doc = 66 ['{10}', '{24}']
ERROR pair tags not equal | num_doc = 75 ['{15}', '{24}']
ERROR pair tags not equal | num_doc = 77 ['{27}', '{24}']


<IPython.core.display.Javascript object>

In [91]:
# for num, elem in enumerate(target_data):
#     print(num, elem)

<IPython.core.display.Javascript object>

In [92]:
# for num, elem in enumerate(real_target_data):
#     print(num, elem)

<IPython.core.display.Javascript object>

In [93]:
few_target

[27]

<IPython.core.display.Javascript object>

In [94]:
few_text

['количество мероприятий по популяризации приобретения субъектами малого и среднего предпринимательства российского программного обеспечения по льготной цене. ']

<IPython.core.display.Javascript object>

In [95]:
elem_old

'{27}количество мероприятий по популяризации приобретения субъектами малого и среднего предпринимательства российского программного обеспечения по льготной цене. {27}'

<IPython.core.display.Javascript object>

In [96]:
few_text1 = re.split("{\d+}[ ,!-.]{0,3}{\d+}", elem_old)
few_text1

['{27}количество мероприятий по популяризации приобретения субъектами малого и среднего предпринимательства российского программного обеспечения по льготной цене. {27}']

<IPython.core.display.Javascript object>

In [97]:
target

27

<IPython.core.display.Javascript object>

In [98]:
target_data[:5]

['1. {2} Настоящие Правила устанавливают цели, условия и порядок предоставления субсидии из федерального бюджета Российскому фонду развития информационных технологий на возмещение затрат по использованию субъектами малого и среднего предпринимательства российского программного обеспечения (далее соответственно - Фонд, субсидия). {2}',
 '{2}Субсидия предоставляется в рамках реализации федерального  "Цифровые технологии" национальной  "Цифровая экономика Российской Федерации".{2}',
 '2. {3}Субсидия предоставляется Министерством цифрового развития, связи и массовых коммуникаций Российской Федерации в пределах лимитов бюджетных обязательств, доведенных в установленном порядке до Министерства цифрового развития, связи и массовых коммуникаций Российской Федерации как получателя средств федерального бюджета на цели, указанные в  настоящих Правил. {3}',
 '{7}Сведения о субсидии размещаются на едином портале бюджетной системы Российской Федерации в информационно-телекоммуникационной сети "Интер

<IPython.core.display.Javascript object>

In [83]:
texts[:5]

['Настоящие Правила устанавливают цели, условия и порядок предоставления субсидий из федерального бюджета центрам исторического наследия президентов Российской Федерации, прекративших исполнение своих полномочий (далее соответственно - субсидии, центры, получатель субсидии).',
 'Субсидии предоставляются центрам, созданным в соответствии с Федеральным  "О центрах исторического наследия президентов Российской Федерации, прекративших исполнение своих полномочий", в целях изучения и публичного представления исторического наследия Президента Российской Федерации, прекратившего исполнение своих полномочий, как неотъемлемой части новейшей истории России, развития демократических институтов и построения правового государства.',
 'Предоставление субсидий осуществляется в пределах лимитов бюджетных обязательств, доведенных в установленном порядке до Управления делами Президента Российской Федерации как получателя средств федерального бюджета (далее - главный распорядитель средств федерального бю

<IPython.core.display.Javascript object>

In [84]:
targets[:5]

[2, 2, 3, 0, 30]

<IPython.core.display.Javascript object>